<a href="https://colab.research.google.com/github/alkhey130/Agentimobilier/blob/main/miniPROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers peft datasets accelerate



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# 1) Modèle simple et stable
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2) Petit dataset "service client"
data = [
    {"text": "Client: Bonjour, j'ai un problème avec ma commande.\nAgent: Bonjour, pouvez-vous me fournir votre numéro de commande ?"},
    {"text": "Client: Ma livraison est en retard.\nAgent: Pouvez-vous me donner votre numéro de suivi ?"},
    {"text": "Client: Je veux annuler ma commande.\nAgent: D’accord, je peux vous aider. Quel est votre numéro de commande ?"}
]

dataset = Dataset.from_list(data)

def preprocess(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

dataset = dataset.map(preprocess)

# 3) Configuration LoRA simple
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["attn.c_attn", "attn.c_proj"],  # Pour GPT-2
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# 4) Entraînement
training_args = TrainingArguments(
    output_dir="./sc_lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=False,
    logging_steps=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()

model.save_pretrained("lora_service_client")


Map: 100%|██████████| 3/3 [00:00<00:00, 21.21 examples/s]
c:\Users\ALI\AppData\Local\Programs\Python\Python313\Lib\site-packages\peft\tuners\lora\layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
c:\Users\ALI\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,6.769100
2,6.658100


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("gpt2")
base = AutoModelForCausalLM.from_pretrained("gpt2")

model = PeftModel.from_pretrained(base, "lora_service_client")

prompt = "Client: Bonjour, j’ai bien récu ma commande merci.\nAgent:"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Client: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai bien récu ma commande merci.
Agent: Bonjour, j’ai b
